In [ ]:
#| label: filtFig7jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path
import os
import sys
from contextlib import contextmanager
from repo2data.repo2data import Repo2Data

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(__file__).parents[2]
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)

data_dir = dataset_path / "04-B1-03-Filtering"
data_file = "b1filt_fig4.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

b1_func = mat_contents["b1_func"][0]
step_b1= mat_contents["step_b1"][0]
gauss_b1_1d = mat_contents["gauss_b1_1d"]
median_b1_1d = mat_contents["median_b1_1d"]
spline_b1_1d = mat_contents["spline_b1_1d"]
vox_range = mat_contents["vox_range"][0]

## Plot


import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

# PYTHON CODE

init_notebook_mode(connected=True)


## Setup for plots
fig = make_subplots(rows=2, cols=2, vertical_spacing = 0.06,
                    subplot_titles=(
    '<b>Unfiltered</b>',
    '<b>Gaussian</b>',
    '<b>Median</b>',
    '<b>Spline</b>',))



position = np.arange(1,len(b1_func)+1)

step_signal = [dict(
            visible = False,
            x = position,
            y = step_b1,
            name = "Step signal",
            hoverinfo = "y",
            showlegend=False) for ii in range(len(vox_range))]

step_signal[2]['visible'] = True
step_signal[2]['showlegend'] = True

gauss_b1 = [dict(
            visible = False,
            x = position,
            y = list(gauss_b1_1d[ii]),
            name = "Gaussian filtered",
            hoverinfo = "y",
            showlegend=False) for ii in range(len(vox_range))]

gauss_b1[2]['visible'] = True
gauss_b1[2]['showlegend'] = True

median_b1 = [dict(
            visible = False,
            x = position,
            y = list(median_b1_1d[ii]),
            name = "Median filtered",
            hoverinfo = "y",
            showlegend=False) for ii in range(len(vox_range))]

median_b1[2]['visible'] = True
median_b1[2]['showlegend'] = True

spline_b1 = [dict(
            visible = False,
            x = position,
            y = list(spline_b1_1d[ii]),
            name = "Spline interpolation",
            hoverinfo = "y",
            showlegend=False) for ii in range(len(vox_range))]

spline_b1[2]['visible'] = True
spline_b1[2]['showlegend'] = True

# Add traces
for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(step_signal[ii], line=dict(color="black", width=2)), row= 1, col=1)

step_signal[2]['showlegend'] = False

for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(gauss_b1[ii], line=dict(color="red", width=3)), row= 1, col=2)
for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(step_signal[ii], line=dict(color="black", width=1)), row= 1, col=2)

for ii in range(len(vox_range)):        
    fig.add_trace(go.Scatter(median_b1[ii], line=dict(color="blue", width=3)), row= 2, col=1)
for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(step_signal[ii], line=dict(color="black", width=1)), row= 2, col=1)
    
for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(spline_b1[ii], line=dict(color="green", width=3)), row= 2, col=2)
for ii in range(len(vox_range)):
    fig.add_trace(go.Scatter(step_signal[ii], line=dict(color="black", width=1)), row= 2, col=2)
    
# Create and add slider
steps = []
      
for i in range(len(vox_range)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data), "showlegend": [False] * len(fig.data)},],  # layout attribute
        label = str(i+1)
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+1*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+2*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+3*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+4*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+5*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+6*len(vox_range)] = True  # Toggle i'th trace to "visible"

    step["args"][0]["showlegend"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["showlegend"][i+1*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["showlegend"][i+3*len(vox_range)] = True  # Toggle i'th trace to "visible"
    step["args"][0]["showlegend"][i+5*len(vox_range)] = True  # Toggle i'th trace to "visible"

    steps.append(step)

sliders = [dict(
    active=2,
    currentvalue={"prefix": "Filter strength: "},
    pad={"t": 50},
    steps=steps
)]

layout = go.Layout(
    width=750,
    height=750,
    margin=go.layout.Margin(
        l=100,
        r=80,
        b=100,
        t=130,
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.25,
        y=1.3,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig.update_layout(layout)

iplot(fig, filename = 'fig4.html', config = config)
